In [13]:
from vanna.openai import OpenAI_Chat
from vanna.chromadb import ChromaDB_VectorStore

In [14]:
import json
import openai

# Load the config file
with open("config.json", "r") as config_file:
    config = json.load(config_file)

# Retrieve and set the API key
openai.api_key = config.get("OPENAI_API_KEY")
postgres_password = config.get("POSTGRES_PASSWORD")
if not openai.api_key:
    raise ValueError("OPENAI_API_KEY is not set in config.json!")


In [16]:
pair_format = {
    "question": "what are daily averages for max battery power and min battery power differences for each Imei?",
    "sql": 'SELECT "Imei", AVG(Daily_Diff_Battery_Power) AS Daily_Avg_Diff_Battery_Power FROM (SELECT "Imei", DATE("Timestamp") AS Date MAX("BMS_UBatt") - MIN("BMS_UBatt") AS Daily_Diff_Battery_Power FROM "Catalog"."DeviceTelemetryData" GROUP BY "Imei", Date) AS DailyDiffs GROUP BY "Imei";'}

column_names = [
    "LastModifiedOn""Id""Latitude""Longitude""BMS_IBatt""BMS_UBatt""BMS_UserSOC""BMS_InternalSOC""BMS_UCellAvg""BMS_UCellMax""BMS_UCellMin""BMS_CellTMax""BMS_CellTMin""BMS_SOCRecalibState""Timestamp""CreatedBy""CreatedOn""LastModifiedBy""Imei"
]


class MyVanna(ChromaDB_VectorStore, OpenAI_Chat):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        OpenAI_Chat.__init__(self, config=config)
    
    def generate_queries(self, sql: str):
        my_prompt = [
            self.system_message(f"You are a helpful assistant that will generate advanced question sql pairs using column names: {column_names} with following format: {pair_format}"),
            self.user_message("Create queries based on the information schema and tables you have.")
        ]

        return self.submit_prompt(prompt = my_prompt)

vn = MyVanna(config={
    'api_key': openai.api_key,
    'model': 'gpt-4'
})


In [17]:
vn.connect_to_postgres(host='localhost',
                        dbname='devicetelemetry',
                        user='postgres', 
                        password=postgres_password,
                        port='5432')

In [18]:
df_information_schema = vn.run_sql("SELECT * FROM INFORMATION_SCHEMA.COLUMNS")


In [19]:
df_information_schema

,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,...,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
0,devicetelemetry,pg_catalog,pg_collation,collencoding,7,None,NO,integer,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
1,devicetelemetry,pg_catalog,pg_amop,amoplefttype,3,None,NO,oid,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
2,devicetelemetry,pg_catalog,pg_amop,amoprighttype,4,None,NO,oid,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
3,devicetelemetry,pg_catalog,pg_amop,amopstrategy,5,None,NO,smallint,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
4,devicetelemetry,pg_catalog,pg_amop,amoppurpose,6,None,NO,"""char""",NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2087,devicetelemetry,information_schema,routines,external_language,40,None,YES,character varying,NaN,1.073742e+09,...,NO,None,None,None,None,None,NO,NEVER,None,NO
2088,devicetelemetry,pg_catalog,pg_statio_all_sequences,schemaname,2,None,YES,name,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,NO
2089,devicetelemetry,pg_catalog,pg_statio_all_sequences,relname,3,None,YES,name,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,NO
2090,devicetelemetry,information_schema,routines,parameter_style,41,None,YES,character varying,NaN,1.073742e+09,...,NO,None,None,None,None,None,NO,NEVER,None,NO


In [20]:
plan = vn.get_training_plan_generic(df_information_schema)
plan

Train on Information Schema: devicetelemetry.pg_catalog pg_collation
Train on Information Schema: devicetelemetry.pg_catalog pg_amop
Train on Information Schema: devicetelemetry.pg_catalog pg_amproc
Train on Information Schema: devicetelemetry.pg_catalog pg_language
Train on Information Schema: devicetelemetry.pg_catalog pg_largeobject_metadata
Train on Information Schema: devicetelemetry.pg_catalog pg_largeobject
Train on Information Schema: devicetelemetry.pg_catalog pg_aggregate
Train on Information Schema: devicetelemetry.pg_catalog pg_statistic
Train on Information Schema: devicetelemetry.pg_catalog pg_statistic_ext
Train on Information Schema: devicetelemetry.pg_catalog pg_statistic_ext_data
Train on Information Schema: devicetelemetry.pg_catalog pg_rewrite
Train on Information Schema: devicetelemetry.pg_catalog pg_trigger
Train on Information Schema: devicetelemetry.pg_catalog pg_event_trigger
Train on Information Schema: devicetelemetry.pg_catalog pg_description
Train on Inform

In [21]:
# vn.train(plan=plan)

In [22]:
# vn.ask(question="What are the ID's and battery powers from DeviceTelemetryData")


In [23]:
vn.generate_queries("Give other question sql pairs.")

Using model gpt-4 for 197.5 tokens (approx)


'Sure, here are the queries:\n\n1. \n{\'question\': \'What is the current state of battery recalibration for each device?\', \n\'sql\': \'SELECT "Imei", "BMS_SOCRecalibState" FROM "Catalog"."DeviceTelemetryData" GROUP BY "Imei";\'}\n\n2. \n{\'question\': \'What is the average internal state of charge (SOC) for each device?\', \n\'sql\': \'SELECT "Imei", AVG("BMS_InternalSOC") AS Average_Internal_SOC FROM "Catalog"."DeviceTelemetryData" GROUP BY "Imei";\'}\n\n3. \n{\'question\': \'What are the maximum and minimum cell temperatures for each device?\', \n\'sql\': \'SELECT "Imei", MAX("BMS_CellTMax"), MIN("BMS_CellTMin") FROM "Catalog"."DeviceTelemetryData" GROUP BY "Imei";\'}\n\n4. \n{\'question\': \'What is the last modified date for each device?\', \n\'sql\': \'SELECT "Imei", MAX("LastModifiedOn") FROM "Catalog"."DeviceTelemetryData" GROUP BY "Imei";\'}\n\n5. \n{\'question\': \'What is the average latitude and longitude for each device?\', \n\'sql\': \'SELECT "Imei", AVG("Latitude"), AV

In [24]:
pair_format_1 = {
    "question": "What is the current state of battery recalibration for each device?",
    "sql": 'SELECT "Imei", "BMS_SOCRecalibState" FROM "Catalog"."DeviceTelemetryData" GROUP BY "Imei";'
}

pair_format_2 = {
    "question": "What is the average internal state of charge (SOC) for each device?",
    "sql": 'SELECT "Imei", AVG("BMS_InternalSOC") AS Average_Internal_SOC FROM "Catalog"."DeviceTelemetryData" GROUP BY "Imei";'
}

pair_format_3 = {
    "question": "What are the maximum and minimum cell temperatures for each device?",
    "sql": 'SELECT "Imei", MAX("BMS_CellTMax"), MIN("BMS_CellTMin") FROM "Catalog"."DeviceTelemetryData" GROUP BY "Imei";'
}

pair_format_4 = {
    "question": "What is the last modified date for each device?",
    "sql": 'SELECT "Imei", MAX("LastModifiedOn") FROM "Catalog"."DeviceTelemetryData" GROUP BY "Imei";'
}

pair_format_5 = {
    "question": "What is the average latitude and longitude for each device?",
    "sql": 'SELECT "Imei", AVG("Latitude"), AVG("Longitude") FROM "Catalog"."DeviceTelemetryData" GROUP BY "Imei";'
}

pair_format_6 = {
    "question": "What is the maximum battery voltage for each device?",
    "sql": 'SELECT "Imei", MAX("BMS_UBatt") FROM "Catalog"."DeviceTelemetryData" GROUP BY "Imei";'
}

pair_format_7 = {
    "question": "Who was the last person to modify each device?",
    "sql": 'SELECT "Imei", "LastModifiedBy" FROM "Catalog"."DeviceTelemetryData" GROUP BY "Imei";'
}

pair_format_8 = {
    "question": "What is the difference between the maximum and minimum battery voltage for each device?",
    "sql": 'SELECT "Imei", (MAX("BMS_UBatt") - MIN("BMS_UBatt")) as Voltage_Difference FROM "Catalog"."DeviceTelemetryData" GROUP BY "Imei";'
}

vn.train(
    question="What is the current state of battery recalibration for each device?",
    sql='SELECT "Imei", "BMS_SOCRecalibState" FROM "Catalog"."DeviceTelemetryData" GROUP BY "Imei";'
)

vn.train(
    question="What is the average internal state of charge (SOC) for each device?",
    sql='SELECT "Imei", AVG("BMS_InternalSOC") AS Average_Internal_SOC FROM "Catalog"."DeviceTelemetryData" GROUP BY "Imei";'
)

vn.train(
    question="What are the maximum and minimum cell temperatures for each device?",
    sql='SELECT "Imei", MAX("BMS_CellTMax"), MIN("BMS_CellTMin") FROM "Catalog"."DeviceTelemetryData" GROUP BY "Imei";'
)

vn.train(
    question="What is the last modified date for each device?",
    sql='SELECT "Imei", MAX("LastModifiedOn") FROM "Catalog"."DeviceTelemetryData" GROUP BY "Imei";'
)

vn.train(
    question="What is the average latitude and longitude for each device?",
    sql='SELECT "Imei", AVG("Latitude"), AVG("Longitude") FROM "Catalog"."DeviceTelemetryData" GROUP BY "Imei";'
)

vn.train(
    question="What is the maximum battery voltage for each device?",
    sql='SELECT "Imei", MAX("BMS_UBatt") FROM "Catalog"."DeviceTelemetryData" GROUP BY "Imei";'
)

vn.train(
    question="Who was the last person to modify each device?",
    sql='SELECT "Imei", "LastModifiedBy" FROM "Catalog"."DeviceTelemetryData" GROUP BY "Imei";'
)

vn.train(
    question="What is the difference between the maximum and minimum battery voltage for each device?",
    sql='SELECT "Imei", (MAX("BMS_UBatt") - MIN("BMS_UBatt")) as Voltage_Difference FROM "Catalog"."DeviceTelemetryData" GROUP BY "Imei";'
)



'c6208aa8-cbb4-551a-b258-953529283da5-sql'

In [25]:
from vanna.flask import VannaFlaskApp
app = VannaFlaskApp(vn, allow_llm_to_see_data=True)
app.run()

Your app is running at:
http://localhost:8084
 * Serving Flask app 'vanna.flask'
 * Debug mode: on


Using model gpt-4 for 4286.5 tokens (approx)
Using model gpt-4 for 133.5 tokens (approx)
Using model gpt-4 for 4286.5 tokens (approx)
